In [1]:
from appgeopy import *
from my_packages import *

In [2]:
# subroutine 1
search_station_func = lambda well_code: reverse_wellcode2station.get(well_code, np.nan)

# subroutine 2
decode_func = lambda ele: ele.decode("utf-8")


# subroutine 3
def transform_to_df(data_dict, date_key="date", value_key="values"):
    date_arr = pd.to_datetime([decode_func(ele) for ele in data_dict[date_key]])
    value_arr = data_dict[value_key]
    output_timeseries = pd.Series(data=value_arr, index=date_arr)
    return output_timeseries

#### Monthly Groundwater

previous version

current version (2025/5/4)

#### Monthly Rainfall

The monthly rainfall data was prepared in file `20250317_PrepareRainfall.ipynb`

Summarized Workflow
```
The `process_measure_data` function processes precipitation data for a specific station through the following workflow:

1. It extracts data for the selected station and ensures hourly frequency with missing values filled.
2. It calculates daily precipitation totals by summing hourly values and extracts both the summed values and corresponding dates.
3. Similarly, it computes monthly precipitation totals by summing to monthly frequency and extracts the monthly values with their dates.
4. Finally, it organizes all processed data (daily and monthly values with their corresponding dates) into a dictionary that serves as the function's output.
```

#### Monthly Electricity Consumption

#### Monthly Vertical Cumulative Displacements

In [18]:
fpath = r"MonthlyResample_dU_timeseries.pkl"
df = pd.read_pickle(fpath, compression='zip')

info_df = pd.read_pickle(r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\1_PrepareDatasets\3_InSAR\ASC_DESC_inc_azim.xz")
xtwd97_list = info_df['geometry'].apply(lambda ele: ele.x).values
ytwd97_list = info_df['geometry'].apply(lambda ele: ele.y).values

for coord, col in zip([ytwd97_list, xtwd97_list], ["Y_TWD97", "X_TWD97"]):
    df.insert(loc=0, column=col, value=coord)

df = df.reset_index(drop=False)
df = df.rename({"index":"PointKey"}, axis=1)

df.columns = [pd.to_datetime(col[1:]) if col.startswith("D") else col for col in df.columns]

df = df.set_index(["PointKey", "X_TWD97", "Y_TWD97"])

df.columns = pd.to_datetime(df.columns)

output_df = df.loc[:, "2014":"2021"].copy()

#############################################################################
#
# [UPDATED: 2025-04-06] : convert cumdisp to diff between two consecutive months
#
#############################################################################
output_df = output_df.diff(axis=1)
output_df.iloc[:, 0] = df.iloc[:, 0]

#############################################################################
output_df.columns = ["N"+col.strftime("%Y%m%d") for col in output_df.columns]
output_df = output_df.reset_index(drop=False)
output_df.to_pickle(r"1_Input_DataSets/Monthly_DISPLACEMENT_dU_CRFP.xz")